In [1]:
import pandas as pd
import json

# Load the dataset
df = pd.read_csv("emergency_service_routing_with_timestamps.csv")

# Parse timestamp column if it's not already parsed
df['Timestamp'] = pd.to_datetime(df['Timestamp'])

# Extract month-year for time trend analysis
df['MonthYear'] = df['Timestamp'].dt.to_period('M').astype(str)

# Chart 1: Incident Type vs Count
incident_type_counts = (
    df.groupby(['Region_Type', 'Incident_Type'])
    .size()
    .reset_index(name='Count')
    .to_dict(orient='records')
)

# Chart 2: Severity Horizontal Bar
severity_counts = (
    df.groupby(['Region_Type', 'Incident_Type', 'Incident_Severity'])
    .size()
    .reset_index(name='Count')
    .to_dict(orient='records')
)

# Chart 3: Incident trends over time
incident_trends = (
    df.groupby(['Region_Type', 'Incident_Type', 'MonthYear'])
    .size()
    .reset_index(name='Count')
    .to_dict(orient='records')
)

# Save as JSON files
with open('incident_type_counts.json', 'w') as f1:
    json.dump(incident_type_counts, f1, indent=2)

with open('severity_counts.json', 'w') as f2:
    json.dump(severity_counts, f2, indent=2)

with open('incident_trends.json', 'w') as f3:
    json.dump(incident_trends, f3, indent=2)
